In [1]:
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt

In [2]:
%cd utils
from sim_functions import optimize_measurement
%cd ..

C:\Users\User\msc_project\Image-processing-strategies\MRsim\utils
C:\Users\User\msc_project\Image-processing-strategies\MRsim


In [3]:
# Edit the echo time (or list of echo times) and the flip angle:
TE_list = [0.008, 0.0095]#, 0.011, 0.0125, 0.014, 0.0155]
flip_ang = 24
# The last requirement for the code is the dimensions, which can be easily acquired from the volume you work with
# The dimensions needed because we will create a 4D array, where we will store the echo time data and we want to maintain the initial 3D dimensions.
data = nib.load("data/ratatouille_crop.nii.gz")
np_data = data.get_fdata()
dimensions = np.array(np_data.shape)
dimensions

array([109, 129, 218])

In [4]:
# Loading the data
pd= nib.load("data/pd_volume_crop.nii.gz")
pd_vol = pd.get_fdata()
t2s = nib.load("data/t2s_volume_crop.nii.gz")
t2s_vol = t2s.get_fdata()
deltaB0 = nib.load("data/fieldmap_crop.nii.gz")
dB0 = deltaB0.get_fdata()
magnitude, phase = optimize_measurement(pd_vol, t2s_vol,dimensions, dB0,flip_ang,TE_list) # By default, B0 is 3 Tesla

Starting optimize_measurement
Processing TE[0] = 0.008
Starting optimized_signal
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[1] = 0.0095
Starting optimized_signal
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Finished optimize_measurement


In [5]:
# Finally we save the magnitude and the phase to analyze them
temp_mag = nib.Nifti1Image(magnitude, affine=data.affine)
temp_ph = nib.Nifti1Image(phase, affine=data.affine)
path_mag = os.path.join('output','sim_mag.nii.gz')
path_ph = os.path.join('output','sim_phase.nii.gz')
nib.save(temp_mag, path_mag)
nib.save(temp_ph, path_ph)